# Ex2 - Getting and Knowing your Data

Check out [Chipotle Exercises Video Tutorial](https://www.youtube.com/watch?v=lpuYZ5EUyS8&list=PLgJhDSE2ZLxaY_DigHeiIDC1cD09rXgJv&index=2) to watch a data scientist go through the exercises

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import time
import polars as pl

start_time = time.time()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

### Step 3. Assign it to a variable called chipo.

In [2]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
    
chipo = pl.read_csv(url, separator = '\t', null_values=['NULL'])

### Step 4. See the first 10 entries

In [3]:
chipo.head(10)

order_id,quantity,item_name,choice_description,item_price
i64,i64,str,str,str
1,1,"""Chips and Fres…",null,"""$2.39 """
1,1,"""Izze""","""[Clementine]""","""$3.39 """
1,1,"""Nantucket Nect…","""[Apple]""","""$3.39 """
1,1,"""Chips and Toma…",null,"""$2.39 """
2,2,"""Chicken Bowl""","""[Tomatillo-Red…","""$16.98 """
3,1,"""Chicken Bowl""","""[Fresh Tomato …","""$10.98 """
3,1,"""Side of Chips""",null,"""$1.69 """
4,1,"""Steak Burrito""","""[Tomatillo Red…","""$11.75 """
4,1,"""Steak Soft Tac…","""[Tomatillo Gre…","""$9.25 """


### Step 5. What is the number of observations in the dataset?

In [4]:
# Solution 1

chipo.shape  # entries <= 4622 observations

(4622, 5)

In [5]:
# Solution 2

chipo.height # entries <= 4622 observations

4622

### Step 6. What is the number of columns in the dataset?

In [6]:
chipo.width

5

### Step 7. Print the name of all the columns.

In [7]:
chipo.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### Step 8. What are the datatypes of the dataframe?

In [8]:
chipo.schema

{'order_id': Int64,
 'quantity': Int64,
 'item_name': Utf8,
 'choice_description': Utf8,
 'item_price': Utf8}

### Step 9. Which was the most-ordered item? 

In [9]:
chipo.groupby('item_name').agg(
    pl.col('quantity').sum()
).sort('quantity', descending=True).head(1)

item_name,quantity
str,i64
"""Chicken Bowl""",761


### Step 11. What was the most ordered item in the choice_description column?

In [10]:
(chipo.lazy()
    .drop_nulls(subset='choice_description')
    .groupby('choice_description')
    .agg(pl.col('quantity').sum())
    .top_k(1, by='quantity')
    .collect()
)

choice_description,quantity
str,i64
"""[Diet Coke]""",159


### Step 12. How many items were orderd in total?

In [11]:
chipo['quantity'].sum()

4972

### Step 13. Turn the item price into a float

#### Step 13.a. Check the item price type

In [12]:
display(chipo['item_price'].dtype)

#or

display(chipo.schema['item_price'])

Utf8

Utf8

#### Step 13.b. Parse the `item_price` as a `pl.Float32`

In [13]:
chipo = chipo.with_columns(
    pl.col('item_price').str.strip('$ ').cast(pl.Float32)
)

chipo.head()

order_id,quantity,item_name,choice_description,item_price
i64,i64,str,str,f32
1,1,"""Chips and Fres…",null,2.39
1,1,"""Izze""","""[Clementine]""",3.39
1,1,"""Nantucket Nect…","""[Apple]""",3.39
1,1,"""Chips and Toma…",null,2.39
2,2,"""Chicken Bowl""","""[Tomatillo-Red…",16.98


#### Step 13.c. Check the item price type

In [14]:
chipo['item_price'].dtype

Float32

### Step 14. How much was the revenue for the period in the dataset?

In [15]:
revenue = chipo.select(
    (pl.col('quantity') * pl.col('item_price')).sum().round(2)
).item()

print(f'Revenue was: ${revenue}')

Revenue was: $39237.02


### Step 15. How many orders were made in the period?

In [16]:
chipo['order_id'].n_unique()

1834

### Step 16. What is the average revenue amount per order?

In [17]:
chipo.groupby('order_id').agg(
    (pl.col('quantity') * pl.col('item_price')).sum()
).mean().item(row=0, column='quantity')

21.39423104265914

### Step 17. How many different items are sold?

In [18]:
chipo['item_name'].n_unique()

50

In [19]:
print(f"Time elapsed: {time.time() - start_time}")

Time elapsed: 0.23923397064208984
